# Solving the generalized Poisson equation

Here we solve the generalized Poisson equation for the electrostatic potential $\Phi$

$$ \nabla [ \sigma (\nabla \Phi) ] = 0, $$

for an inhomogeneous conductivity $\sigma(\vec{r})$ in 2D. Note that for a homogeneous conductivity, this reduces to the ordinary Poisson equation $\Delta \Phi=0$.

Make sure that the right files are imported:

In [ ]:
try:
    import google.colab
except:
    %run poisson
else:
    !git clone https://github.com/jeroenmulkers/currentpaths
    %run currentpaths/poisson

## Example 1: Homogeneous conductivity

In [ ]:
# Construct a 'Poisson object'
p = Poisson( nx=60, ny=60, xrange=[0.,1.], yrange=[0.,1.], sigma=1.)

# Add a first contact point with a certain voltage
region = lambda x,y: 0.15>x>0.1 and 0.4<y<0.6
p.setPotential(regionFunc=region, U=1.0)

# Of course we need a second contact
region = lambda x,y: 0.9<x<0.95 and 0.4<y<0.6
p.setPotential(regionFunc=region, U=-1.0)

# let's calculate and show the current
p.showCurrent()

## Example 2: high resistivity at the center

In [ ]:
p = Poisson( nx=60, ny=60, xrange=[0.,1.], yrange=[0.,1.], sigma=1.)

# Let's use the same contact points as in example 1
region = lambda x,y: 0.15>x>0.1 and 0.4<y<0.6
p.setPotential(regionFunc=region, U=1.0)
region = lambda x,y: 0.9<x<0.95 and 0.4<y<0.6
p.setPotential(regionFunc=region, U=-1.0)

# Create rectangular region in the center with a low conductivity
# (you can also set it to zero to have an isolating material at the center)
sigmaCenter = 0.3
region = lambda x,y: 0.4<x<0.6 and 0.3<y<0.7
p.setSigma(regionFunc=region, sigma=sigmaCenter)

p.showCurrent()